# 🚀 Showcase: 2D to 360° Video Generation

This notebook provides a comprehensive showcase for the 'Beyond the Frame' project. It will guide you through setting up the environment and then offer two distinct ways to demo the technology:

*   **Option 1: Interactive Demo** - Launch a web interface to upload your own videos.
*   **Option 2: Automated Processing** - Automatically download a video from a URL and process it.

**Instructions:**
1. Make sure your runtime is set to use a GPU (`Runtime` > `Change runtime type` > `T4 GPU`).
2. Run the cells in order. The first cell will install Conda and restart the kernel. **This is expected.** After it restarts, run the remaining cells starting from 'Step 2'.

### Step 1: Install Conda and Restart Kernel

Run the cell below to install `conda` in your Colab environment. **This will restart the Colab kernel automatically.** After the restart, you can proceed to the next cells.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

### Step 2: Common Setup (Run After Kernel Restart)

In [ ]:
print('Cloning repository...')
!git clone --recurse-submodules https://github.com/Red-Fairy/argus-code
%cd argus-code

print('Installing dependencies...')
# Use conda to install faiss-gpu, which is more reliable than pip for this package.
!conda install -y -c pytorch -c nvidia faiss-gpu=1.8.0

# Install other requirements
!pip install -q -r requirements.txt
!pip install -q yt-dlp gdown

print('Cloning and installing asmk...')
!git clone https://github.com/jenicek/asmk
%cd asmk/cython/
!cythonize *.pyx
%cd ..
!pip install .
%cd ..
print('All dependencies installed.')

In [ ]:
print('Downloading pre-trained models...')
!mkdir -p checkpoints
!gdown --id 1mZpViQY2yvwav-CcxdsQouYu8t52b25G -O checkpoints/pretrained-weights.zip
!unzip -q checkpoints/pretrained-weights.zip -d checkpoints/

!mkdir -p mega-sam/Depth-Anything/checkpoints
!wget -q https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth -O mega-sam/Depth-Anything/checkpoints/depth_anything_vitl14.pth

!mkdir -p mega-sam/cvd_opt
!gdown --id 1sWDsfuZ3Up38EUQt7-JDTT1HcGHuJgvT -O mega-sam/cvd_opt/raft-things.pth
print('Models downloaded successfully.')

### Step 3: Choose Your Demo

**Important:** Only run the cells for the option you want to use.

---

#### Option 1: Interactive Demo (Gradio Web UI)

Run the cell below to launch a Gradio web interface. A public URL will be generated. Click it to open the demo in a new tab, where you can upload your own video for processing.

In [ ]:
!python gradio_demo.py --share

---

#### Option 2: Automated Processing from a URL

Run the two cells below to automatically download a video from a URL and process it using the command-line script.

In [ ]:
# --- You can change this URL to any video you want to process ---
video_url = 'https://www.youtube.com/watch?v=YE7VzlLtp-4' # Big Buck Bunny (placeholder)

!mkdir -p input_videos
!yt-dlp -o 'input_videos/video.mp4' -f 'best[height<=720]' {video_url}

video_path = 'input_videos/video.mp4'
print(f'Video downloaded to: {video_path}')

In [ ]:
print('Starting automated processing...')
unet_path = './checkpoints/pretrained-weights'
save_folder = './outputs'

!python inference.py \
    --unet_path {unet_path} \
    --video_path {video_path} \
    --val_save_folder {save_folder} \
    --guidance_scale 3 \
    --num_inference_steps 25

print(f'\n--- Processing Complete! ---\n')
print(f'Check the `{save_folder}` directory for your generated 360° video.')